- 참고 사이트
    https://blog.naver.com/PostView.naver?blogId=beyondlegend&logNo=223050797442

In [43]:
import datetime, cv2, ultralytics
from ultralytics import YOLO

In [44]:
# 커스텀 데이터에 맞는 YAML 파일 만들기 
import yaml

# 저장된 데이터 경로로 변경, 클래스 이름 및 수 변경

data = {'train':"F:/찐막_data/train",
        'val':"F:/찐막_data/valid",
        'test': "F:/찐막_data/test",
        'names':['Person','Trolly','Forklift(H)','Forklift(D)', 'Forklilft(V)'],
        'nc' : 5}

# 지정된 yaml 파일에 작성하기
with open('mnvision_sample.yaml', 'w') as f:
    yaml.dump(data, f)

# 작성된 yaml 파일 확인하기
with open('mnvision_sample.yaml', 'r') as f:
    yolo_yaml = yaml.safe_load(f)
    display(yolo_yaml)

{'names': ['Person', 'Trolly', 'Forklift(H)', 'Forklift(D)', 'Forklilft(V)'],
 'nc': 5,
 'test': 'F:/찐막_data/test',
 'train': 'F:/찐막_data/train',
 'val': 'F:/찐막_data/valid'}

In [45]:
# YOLOv8 설치 확인
ultralytics.checks()

Ultralytics YOLOv8.2.14  Python-3.8.19 torch-2.2.2 CPU (11th Gen Intel Core(TM) i5-1155G7 2.50GHz)
Setup complete  (8 CPUs, 15.7 GB RAM, 215.3/230.6 GB disk)


In [46]:
# YOLO 모델 로딩
model = YOLO('yolov8n.pt')  # MS COCO dataset 사전학습된 모델 로딩함 

# 사전학습된 모델에 정의된 클래스 확인 
print(type(model.names), len(model.names))
print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

In [47]:
import torch
print(torch.cuda.is_available())
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

False
cpu


In [48]:
# 학습 결과를 사이트..?에 기록하는 모듈
# import comet_ml

# comet_ml.init()

In [49]:
# # Initialize the YOLO model
# model = YOLO("yolov8n.pt")

# # Tune hyperparameters on COCO8 for 30 epochs
# model.tune(data="coco8.yaml", epochs=30, iterations=300, optimizer="AdamW", plots=False, save=False, val=False)

In [53]:
# 커스텀 데이터로 학습하기
# 반드시 yaml을 지정해주어야 한다.
model.train(
    data = 'mnvision_sample.yaml', 
    epochs = 1000, 
    patience = 15, 
    batch = 1, 
    imgsz=720,
    pretrained = False,
    plots = True,
    degrees = 90,
    translate = 0.12431,
    scale = 0.07643,
    shear = 0.0
    )

# patience : 검증 손실이 지정된 에포크 수 도안 개선되지 않으면 조기종료 
# imgsz : 입력 이미지의 크기를 (n, n)으로 조정 

# 하이퍼 파라미터
# lr0: 초기 학습률(learning rate)
# lrf: 최종 학습률(learning rate finder로 결정)
# momentum: 모멘텀 값
# weight_decay: 가중치 감소 정도(L2 규제)
# warmup_epochs: warmup 단계에서의 epoch 수
# warmup_momentum: warmup 단계 모멘텀
# box/cls/dfl: 객체 박스, 클래스, 좌표 loss에 대한 가중치
# hsv_h/s/v: HSV 색공간 데이터 augmentation 하이퍼파라미터
# degrees/translate/scale/shear: 각도/평행이동/크기조절/전단(shear) augmentation
# perspective: 원근법 augmentation
# flipud/fliplr: 상하/좌우 반전 augmentation
# mosaic: 모자이크 augmentation 확률
# mixup/copy_paste: Mixup/Copy-Paste augmentation 확률

Ultralytics YOLOv8.2.14  Python-3.8.19 torch-2.2.2 CPU (11th Gen Intel Core(TM) i5-1155G7 2.50GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=mnvision_sample.yaml, epochs=1000, time=None, patience=15, batch=1, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train62, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning F:\찐막_data\train.cache... 831 images, 0 backgrounds, 0 corrupt: 100%|██████████| 831/831 [00:00<?, ?it/s]
val: Scanning F:\찐막_data\valid.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]

Plotting labels to runs\detect\train62\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 736 train, 736 val
Using 0 dataloader workers
Logging results to runs\detect\train62
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     1/1000         0G       1.13      1.413        1.3          4        736: 100%|██████████| 831/831 [07:20<00:00,  1.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 40/40 [00:11<00:00,  3.40it/s]

                   all         80        116      0.849      0.721      0.785      0.266



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     2/1000         0G       1.18       1.37      1.326          2        736:  12%|█▏        | 102/831 [00:54<06:29,  1.87it/s]


KeyboardInterrupt: 

In [51]:
# 커스텀 데이터로 학습된 모델의 클래스 확인 
print(type(model.names), len(model.names))
print(model.names)

<class 'dict'> 5
{0: 'Person', 1: 'Trolly', 2: 'Forklift(H)', 3: 'Forklift(D)', 4: 'Forklilft(V)'}


In [52]:
# 검증하기 
import metrics
model.val()

# 검증 결과 출력 
# print(metrics.box.map)    # mAP (mean Average Precision)
# print(metrics.box.map50)  # mAP@0.5
# print(metrics.box.map75)  # mAP@0.75

ModuleNotFoundError: No module named 'metrics'

In [ ]:
# 커스텀데이터로 학습한 모델 불러온 후 저장하기

# num = 8
# folder_name = f'train{num}'
# model = YOLO(f'runs/detect/{folder_name}/weights/best.pt')
# print('불러오기 완료')

In [ ]:
# onnx 포맷으로 모델 저장하기
# model.export(format='onnx', dynamic=True)  # ONNX 포맷으로 저장
# print('저장완료')

Ultralytics YOLOv8.2.16  Python-3.8.19 torch-2.2.2 CPU (12th Gen Intel Core(TM) i7-12700)

PyTorch: starting from 'runs\detect\train16\weights\best.pt' with input shape (1, 3, 736, 736) BCHW and output shape(s) (1, 9, 11109) (6.0 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success  0.6s, saved as 'runs\detect\train16\weights\best.onnx' (11.5 MB)

Export complete (2.3s)
Results saved to C:\Users\DELL-PC\Desktop\YimSoYoung\try_03_class_05\runs\detect\train16\weights
Predict:         yolo predict task=detect model=runs\detect\train16\weights\best.onnx imgsz=736  
Validate:        yolo val task=detect model=runs\detect\train16\weights\best.onnx imgsz=736 data=mnvision.yaml  
Visualize:       https://netron.app
저장완료


<hr>

In [ ]:
# 예측하기

# img_file = "01frame_12696.jpg"
# results = model.predict(source = img_file, save=True, conf=0.25)

# source : 예측할 이미지, 비디오 또는 웹캠의 경로나 인덱스를 지정
# save : 예측 결과 저장 여부
# conf : 예측 결과의 신뢰도 임계값을 지정함 

In [ ]:
# 비디오 예측

# video_file = "20231129115714.avi"
# results = model.predict(source= video_file, save=True, conf=0.25)

In [ ]:
# 웹캠 예측

# results = model.predict(source='0', save=True, conf=0.25)  # 0은 기본 웹캠을 의미